# Best optimizer parameters:

## RSI_STOCH:

### BUY ~ 
- RSI: low/high bound: **30/70**, timeperiod: **14**
- STOCH: low/high bound: **15/85**, fastk: **14**, slowk: **3**, slowd: **7**

# Parameter optimization

In [11]:
import sys
import pandas as pd
import numpy as np
from optimizer import Optimizer
from os import environ, remove


# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

# Get configs
configs = ConfigFactory.factory(environ).configs


pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['STOCH', 'RSI']
opt_limit = 100
load = False

optim_dict = {'RSI': {'timeperiod': [18], 'low_bound': [30]},
              'STOCH': {'fastk_period': [6], 'slowk_period': [2],
                        'slowd_period': [2], 'low_bound': [25]}}
opt = Optimizer(pattern, optim_dict, **configs)

stat = opt.optimize(pattern, ttype, opt_limit, load)
try:
    open(f'opt_{"_".join(pattern)}_{ttype}.pkl', 'w').close()
except FileNotFoundError:
    pass
stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}.pkl')

stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat = stat.sort_values(['pct_right_forecast_avg', 'forecasts_num'], ascending=False)
stat.head(20)

Number of combinations is 1


  0%|          | 0/1 [00:00<?, ?it/s]

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg
0,STOCH_RSI,18,30,6,2,2,25,81.82,86.36,81.82,86.36,77.27,72.73,86.36,81.82,86.36,81.82,63.64,63.64,68.18,68.18,68.18,86.36,81.82,81.82,81.82,81.82,86.36,81.82,81.82,86.36,0.32,0.33,0.46,0.53,0.43,0.34,0.51,0.53,0.44,0.5,0.42,0.26,0.57,0.71,0.43,0.47,0.51,0.54,0.65,0.61,0.76,0.96,0.78,0.57,22,79.355833


In [2]:
# objgraph.show_growth()

# from pympler import muppy
# all_objects = muppy.get_objects()
# top_10 = muppy.sort(all_objects)[-10:]
# top1 = top_10[0]
# top1

# tb = tracemalloc.get_object_traceback(top_10[1])
# tb.format()